In [ ]:
# tournament_kagaribi_8_db.xlsxで指定した大会の試合情報をsmash_ggから取得し、出力する

In [1]:
# https://smash.gg/admin/user/782f51a2/developer で取得したsmashggトークン
# プログラム公開時はマスキングすること！

smashgg_token = "xxxxxxxxxxxxxxxxxxxxxxx"

In [2]:
import pysmashgg
import pandas as pd
import time
from datetime import datetime
import swifter
import pickle

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option("display.max_colwidth", 80)

In [10]:
def fetch_tournament_name(tournament_URL):
    """
    大会イベントURLを与えると、URL中の大会名を返す
    """
    separated_url = tournament_URL.split("/")
    return separated_url[separated_url.index("tournament")+1]

In [11]:
def fetch_event_name(tournament_URL):
    """
    大会イベントURLを与えると、URL中のイベント名を返す
    """
    separated_url = tournament_URL.split("/")
    return separated_url[separated_url.index("event")+1]

In [20]:
def fetch_df_all_players(tournament_event_id:int):
    """
    大会イベントIDを与えると、参加した全選手のデータフレームを返す関数
    event_show_setsで試合を取得すると、なぜか取得漏れが発生するのでこの形式にしている
    """

    df_players = pd.DataFrame()
    page = 1 # 開始ページ。なぜか-1などの負の数も指定可(最後の重複除去時に結局同じ結果になる)
    
    # 全ページ巡回
    while 1:
        df_tmp = pd.DataFrame.from_dict(smash.event_show_entrants(tournament_event_id, page))
        if len(df_tmp):
            df_players = pd.concat([df_players, df_tmp])
            page += 1
        else:
            break
    
    # リストを内包しているentrantPlayersカラムから必要な情報を抜き出す
    df_players = (df_players
                  .assign(playerId = lambda x: x.entrantPlayers.swifter.progress_bar(False).apply(lambda y: y[0]["playerId"]))
                  .assign(playerTag = lambda x: x.entrantPlayers.swifter.progress_bar(False).apply(lambda y: y[0]["playerTag"]))
                  .drop(columns="entrantPlayers")
                  .drop_duplicates()
                  .assign(tournament_event_id = tournament_event_id)
                 )

    return df_players

In [21]:
def fetch_df_all_sets(tournament_event_id:int, player_tags:list):
    """
    大会イベントIDと全プレイヤーの"tag"リストを与えると、全試合(set)のデータフレームを返す
    """
    
    df_sets = pd.DataFrame()

    for player_tag in player_tags:
        df_sets = pd.concat([df_sets, pd.DataFrame.from_dict(smash.event_show_entrant_sets(tournament_event_id, player_tag))])

    df_sets = (df_sets
               .drop_duplicates()
               .assign(tournament_event_id = tournament_event_id)
              )
    
    return df_sets

In [14]:
# 取得したsmashggトークンを使ってクラス定義
# 2番目の変数はオートリトライの有無

smash = pysmashgg.SmashGG(smashgg_token, True)

In [22]:
# 大会情報データフレームを作成、出力する
# 大会イベントURLからの大会名、イベント名の取得(URL取得はfetch_tournament_url.ipynbを使用)と、大会イベントIDの取得

df_t = (pd.read_excel("tournament_kagaribi_8_db.xlsx")
        [["tournament_url"]]
        .assign(tournament_name = lambda x: x.tournament_url.swifter.progress_bar(False).apply(fetch_tournament_name))
        .assign(event_name = lambda x: x.tournament_url.swifter.progress_bar(False).apply(fetch_event_name))
        .assign(tournament_event_id = lambda x: x.swifter.progress_bar(False).apply(lambda x: smash.tournament_show_event_id(x.tournament_name, x.event_name), axis=1))
       )

Error 429: Sending too many requests right now, trying again in 10 seconds
Error 429: Sending too many requests right now, trying again in 20 seconds
Error 429: Sending too many requests right now, trying again in 10 seconds
Error 429: Sending too many requests right now, trying again in 20 seconds


In [24]:
# 大会名を使用してsmashggから大会情報を取得

df_tmp = pd.DataFrame()
for t_name in df_t.tournament_name:
    df_tmp = pd.concat([df_tmp, pd.DataFrame(smash.tournament_show(t_name), index=[""])]) # ダミーインデックスが無いとエラー

Error 429: Sending too many requests right now, trying again in 10 seconds
Error 429: Sending too many requests right now, trying again in 20 seconds
Error 429: Sending too many requests right now, trying again in 10 seconds
Error 429: Sending too many requests right now, trying again in 20 seconds


In [25]:
# 大会情報データフレームを加工して横連結

df_tmp = (df_tmp
          .rename(columns={"name":"JPN_name"}) # 後々カラム名が重複するので改名
          .reset_index(drop=True)
          .assign(startday = lambda x: x.startTimestamp.swifter.progress_bar(False).apply(datetime.fromtimestamp).dt.date) # timestampを日付に修正
          .assign(endday = lambda x: x.endTimestamp.swifter.progress_bar(False).apply(datetime.fromtimestamp).dt.date)
          .drop(columns=["startTimestamp", "endTimestamp"])
         )
df_tmp.columns = ["tournament_" + x for x in df_tmp.columns.to_list()] # 大会情報のカラム名にprefixをつける

df_t = pd.concat([df_t, df_tmp], axis=1)
df_t.to_csv("df_t.csv", index=False, encoding="utf-8-sig")
df_t

,tournament_url,tournament_name,event_name,tournament_event_id,tournament_id,tournament_JPN_name,tournament_country,tournament_state,tournament_city,tournament_entrants,tournament_startday,tournament_endday
0,https://smash.gg/tournament/3-kagaribi-3/event/singles/overview,3-kagaribi-3,singles,558662,275805,篝火#3 / Kagaribi#3,JP,東京都,大田区,132,2021-03-27,2021-03-27
1,https://smash.gg/tournament/sp14/event/ishibura-sp14/overview,sp14,ishibura-sp14,562395,263073,いしぶら!SP14,JP,石川県,野々市市,64,2021-04-04,2021-04-04
2,https://smash.gg/tournament/winner-1/event/singles-1/overview,winner-1,singles-1,584890,294706,WINNER!,JP,東京都,豊島区,48,2021-06-21,2021-06-21
3,https://smash.gg/tournament/4-kagaribi-4-1/event/singles/overview,4-kagaribi-4-1,singles,566593,281939,篝火 #4 / Kagaribi #4,JP,東京都,大田区,420,2021-06-26,2021-06-27
4,https://smash.gg/tournament/1on1-top-5-maesumatop-5/event/singles-tournament...,1on1-top-5-maesumatop-5,singles-tournament,588686,297173,【1on1部門】マエスマTOP#5/ MaesumaTOP#5,JP,大阪府,大東市,128,2021-07-04,2021-07-04
...,...,...,...,...,...,...,...,...,...,...,...,...
173,https://www.start.gg/tournament/02-kurobraweekly-02/event/sp-1on1,02-kurobraweekly-02,sp-1on1,825399,502429,クロブラ平日大会02/KurobraWeekly#02,JP,神奈川県,横浜市,28,2022-12-08,2022-12-08
174,https://www.start.gg/tournament/03-kurobraweekly-03/event/sp-1on1,03-kurobraweekly-03,sp-1on1,828681,504302,クロブラ平日大会03/KurobraWeekly#03,JP,神奈川県,横浜市,26,2022-12-15,2022-12-15
175,https://www.start.gg/tournament/delta-1/event/singles/overview,delta-1,singles,814192,496169,DELTA #1,JP,東京都,中野区,71,2022-12-18,2022-12-18
176,https://www.start.gg/tournament/tsb-17/event/special-squad-strike-3-v-3/over...,tsb-17,special-squad-strike-3-v-3,816447,497471,TSB#17,JP,Tokyo,Nakano City,20,2022-11-20,2022-11-20


In [27]:
# 各大会イベントに参加した全プレイヤーを抽出する(｢大会+参加プレイヤー｣ごとにユニークになる点に注意)

df_players = pd.DataFrame()

for tournament_event_id in df_t.tournament_event_id:
    df_players = pd.concat([df_players, fetch_df_all_players(tournament_event_id)])
    
df_players.to_csv("df_players.csv", index=False, encoding="utf-8-sig")
df_players

Error 429: Sending too many requests right now, trying again in 10 seconds
Error 429: Sending too many requests right now, trying again in 20 seconds
Error 429: Sending too many requests right now, trying again in 10 seconds
Error 429: Sending too many requests right now, trying again in 20 seconds
Error 429: Sending too many requests right now, trying again in 10 seconds
Error 429: Sending too many requests right now, trying again in 20 seconds
Error 429: Sending too many requests right now, trying again in 10 seconds
Error 429: Sending too many requests right now, trying again in 20 seconds
Error 429: Sending too many requests right now, trying again in 10 seconds
Error 429: Sending too many requests right now, trying again in 20 seconds
Error 429: Sending too many requests right now, trying again in 10 seconds
Error 429: Sending too many requests right now, trying again in 20 seconds
Error 429: Sending too many requests right now, trying again in 10 seconds
Error 429: Sending too ma

,entrantId,tag,finalPlacement,seed,playerId,playerTag,tournament_event_id
0,7007365,SG8 | Tea,1,6,695882,Tea,558662
1,7007274,PNG | Atelier,2,13,553122,Atelier,558662
2,7007322,アカキクス,3,5,1857350,アカキクス,558662
3,7007280,GW | zackray,4,1,498962,zackray,558662
4,7007633,tk3,5,10,563678,tk3,558662
...,...,...,...,...,...,...,...
18,11641991,Nico,257,369,2229287,Nico,802437
19,11649709,はかせ,257,208,2736310,はかせ,802437
20,11665470,ういろう,257,240,1623014,ういろう,802437
21,11649633,rod | rod,257,273,2898287,rod,802437


In [36]:
# 各大会イベントの参加プレイヤーの全試合を抽出し、結合して出力する。178大会で10h以上かかる
# 742002(https://www.start.gg/tournament/offline-feat/event/singles-tournament/overview)は成功しても試合を取得できない
# サーバーエラーが頻発するようになったらrestart_flgをTrueにしてRestart Kernelからやり直す。根気よくリトライしているとそのうち成功する

# エラー時再開フラグ
restart_flg = False

if restart_flg: # 関数定義セルまで実行したら次はこのセルから実行すればOK
    smash = pysmashgg.SmashGG(smashgg_token, True)
    df_t = pd.read_csv("df_t.csv", encoding="utf-8-sig")
    df_players = pd.read_csv("df_players.csv", encoding="utf-8-sig")
    df_sets = pd.read_csv("df_original_sets.csv", encoding="utf-8-sig")
    tournament_event_id_set = pickle.load(open("tournament_event_id_set.txt","rb"))
else:
    df_sets = pd.DataFrame()
    tournament_event_id_set = set(df_t.tournament_event_id)

while tournament_event_id_set:
    tournament_event_id = tournament_event_id_set.pop()
    try:
        df_sets = (pd.concat([df_sets, 
                              fetch_df_all_sets(tournament_event_id, df_players[df_players.tournament_event_id == tournament_event_id].tag)])
                   .drop_duplicates()
                  )
        df_sets.to_csv("df_original_sets.csv", index=False, encoding="utf-8-sig")
        print("success", tournament_event_id)
    except:
        tournament_event_id_set.add(tournament_event_id)
        print("failure", tournament_event_id)
    pickle.dump(tournament_event_id_set, open('tournament_event_id_set.txt', 'wb'))

In [37]:
# 勝者の結合用データフレーム
df_tmp1 = df_players.copy()
df_tmp1.columns = ["winner_"+x for x in df_tmp1.columns.to_list()]
df_tmp1 = df_tmp1.rename(columns={"winner_tournament_event_id":"tournament_event_id", "winner_entrantId":"winnerId"})

# 敗者の結合用データフレーム
df_tmp2 = df_players.copy()
df_tmp2.columns = ["loser_"+x for x in df_tmp2.columns.to_list()]
df_tmp2 = df_tmp2.rename(columns={"loser_tournament_event_id":"tournament_event_id", "loser_entrantId":"loserId"})

In [45]:
# 大会情報とプレイヤー情報をマージし、時系列順にソート、さらにDQを除いて全試合情報を出力
# 理想的に大会結果が入力されていれば、大会開催日→大会イベントID→bracketId→idの順に昇順に並べると時系列順になる模様
# idが稀に前後しているので、個別に作成したsetRound_orderによるソートを挟む

df_sets = (df_sets
           .merge(df_tmp1, on=["tournament_event_id", "winnerId"])
           .merge(df_tmp2, on=["tournament_event_id", "loserId"])
           .merge(df_t, on="tournament_event_id")
           .merge(pd.read_excel("setRound_sort.xlsx"), on="setRound", how="left")
           .sort_values(["tournament_startday", "tournament_event_id", "bracketId", "setRound_order", "id"])
           .pipe(lambda x: x[(x.entrant1Score >= 0) & (x.entrant2Score >= 0)]) # DQ除外。DQはentrantScoreが-1となる
          )
    
df_sets.to_csv("df_sets.csv", index=False, encoding="utf-8-sig")
df_sets